# CSCA 5622 Intro to Machine Learning: Final Project

## Likelihood of Student Criminal Activity

Using public datasets from the Department of Education, find any correlations between states where children attended preschool, teacher certifications, and criminal offenses ocurring at school.  


#### DATA INFO

**Teacher Credentials:**
- The `teacher_creds` data frame records number and percentage of public school classroom teachers (in full-time equivalents), by certification status and years of experience, by state: School Year 2013-14.

- Table reads (for US Totals): Of all 3,138,535 classroom teachers (FTE), 3,084,697 (98.3%) met all state licensing/certification requirements. Data reported in this table represent 100.0% of responding schools.

- *Source: U.S. Department of Education, Office for Civil Rights, Civil Rights Data Collection, 2013-14, available at http://ocrdata.ed.gov. Data notes are available at http://ocrdata.ed.gov/downloads/DataNotes.docx*
 
**Preschool Enrollment:**
- The `preschool` data frame records number and percentage of public school students enrolled in Preschool, by race/ethnicity, disability status, and English proficiency, by state: School Year 2015-16.

- Table reads (for US Totals): Of all 1,536,982 public school students enrolled in Preschool, 17,964 (1.2%) were American Indian or Alaska Native, and 313,601 (20.4%) were students with disabilities served under the Individuals with Disabilities Education Act (IDEA). Data reported in this table represent 100.0% of responding schools.																						
	
- *Source: U.S. Department of Education, Office for Civil Rights, Civil Rights Data Collection, 2015-16, available at http://ocrdata.ed.gov. Data notes are available at https://ocrdata.ed.gov/Downloads/Data-Notes-2015-16-CRDC.pdf*

**School Incidents:**
- The `incidents` data frame records number of incidents, by state: School Year 2015-16.

- Table reads (for US): The number of incidents of sexual assault was 9,255. Data reported in this table represent 98.0% of responding schools.

- *Source: U.S. Department of Education, Office for Civil Rights, Civil Rights Data Collection, 2015-16, available at http://ocrdata.ed.gov. Data notes are available at https://ocrdata.ed.gov/Downloads/Data-Notes-2015-16-CRDC.pdf.*																	

##### Disclaimers 
- Due to limited data available, the teacher credentials data is from school year 2013-2014 while the other two data sets are from 2015-2016.
- The school incidents data provides a disclaimer at the footer to "Interpret data in this row with caution. Data are missing for more than 15 percent of schools."
- This is only one school year's worth of data and would probably be more accurate with several years. 

**This results of this experiment will inherently be inacurrate so this should not be interpreted as factual.**

In [64]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns

# Set color map to have light blue background
sns.set()
from pathlib import Path

from src.csca5622_final_project.utils import load_excel
from src.csca5622_final_project.column_mappings import PRESCHOOL_ENROLLMENT_COLUMN_MAPPINGS, TEACHER_CREDENTIALS_COLUMN_MAPPINGS, SCHOOL_INCIDENT_COLUMN_MAPPINGS

# Load & Prepare Data

In [65]:
BASE_PATH = Path.cwd().joinpath("../../data/")

## Load & Prepare Teacher Credentials Data
- Load the Excel sheet and exlcude the top few rows of formatting.
- Rename column names. See comments in the code for mappings.
- Remove the empty 0 index `NaN` column from Excel sheet.
- Drop the last few rows of metada and formatting. 

In [66]:
teacher_creds = load_excel(
    file_path="teacher-certification-and-years-of-experience.xlsx",
    names=TEACHER_CREDENTIALS_COLUMN_MAPPINGS,
    drop_columns=["C0"],
)
teacher_creds.head()

,state,classroom_teachers_fte,meeting_requirements_fte,meeting_requirements_p,first_year_teachers_fte,first_year_teachers_p,second_year_teachers_fte,second_year_teachers_p,total_schools,total_schools_p
0,United States,3138534.55,3084697.40,98.2846,216087.56,6.8850,163067.17,5.19565,95507.0,99.971
1,Alabama,47203.01,46839.32,99.2295,2743.64,5.8124,2296.30,4.86473,1397.0,100.000
2,Alaska,7707.29,7639.21,99.1167,506.90,6.5769,304.97,3.95690,495.0,100.000
3,Arizona,56969.03,54072.38,94.9154,4564.63,8.0125,4046.82,7.10354,1913.0,100.000
4,Arkansas,34956.94,34450.44,98.5511,2200.10,6.2937,1822.91,5.21473,1085.0,100.000


## Load & Prepare Preschool Data
- Load theExcel sheet and exlcude the top few rows of formatting.
- Rename column names. See comments in the code for mappings.
- Remove the empty 0 index `NaN` column from Excel sheet.
- Drop the last few rows of metada and formatting.

In [67]:
preschool = load_excel(
    "preschool-enrollment.xlsx",
    names=PRESCHOOL_ENROLLMENT_COLUMN_MAPPINGS,
    drop_columns=["C0"],
)
preschool.head()

/Users/seanbaier/Documents/projects/mscs/Courses/machine-learning/intro-machine-learning/csca5622-final-project/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,state,total,total_percent,american_indian_alaska_native,american_indian_alaska_native_p,asian,asian_p,hispanic_latino_any,hispanic_latino_any_p,black_african_american,...,native_hawaiian_pacific_islander,native_hawaiian_pacific_islander_p,mixed_race,mixed_race_p,students_disabilities_idea,students_disabilities_idea_p,english_language_learners,english_language_learners_p,total_schools,total_schools_p
0,United States,1536982.0,100.0,17964.0,1.1688,56480.0,3.6747,454433.0,29.5666,292372.0,...,3721.0,0.2421,59273.0,3.8565,313601.0,20.4037,194932.0,12.6828,96360.0,99.998
1,Alabama,13803.0,100.0,155.0,1.1229,154.0,1.1157,971.0,7.0347,5655.0,...,15.0,0.1087,354.0,2.5647,2240.0,16.2284,364.0,2.6371,1400.0,100.000
2,Alaska,3249.0,100.0,1400.0,43.0902,87.0,2.6777,164.0,5.0477,79.0,...,74.0,2.2776,374.0,11.5112,1125.0,34.6260,17.0,0.5232,503.0,100.000
3,Arizona,20212.0,100.0,1201.0,5.9420,635.0,3.1417,8717.0,43.1278,783.0,...,52.0,0.2573,667.0,3.3000,7994.0,39.5508,221.0,1.0934,1977.0,100.000
4,Arkansas,15387.0,100.0,48.0,0.3120,147.0,0.9554,2300.0,14.9477,4487.0,...,111.0,0.7214,370.0,2.4046,830.0,5.3942,656.0,4.2633,1092.0,100.000


## Load & Prepare School Incident Data
- Load theExcel sheet and exlcude the top few rows of formatting.
- Rename column names. See comments in the code for mappings.
- Remove the empty 0 index `NaN` column from Excel sheet.
- Drop the last few rows of metada and formatting.

In [68]:
incident_col_mappings = [
    "C0",
    "state",
    # "Rape or attempted rape",
    "rape_atmp_rape",
    # "Sexual assault",
    "sex_aslt",
    # "Robbery with a weapon",
    "rob_weap",
    # "Robbery with a firearm or explosive",
    "rob_fire_expl",
    # "Robbery without a weapon",
    "rob",
    # "Physical attack or fight with a weapon",
    "phys_atk_fight_weap",
    # "Physical attack or fight with a firearm or explosive device",
    "phys_atk_fight_fire_expl",
    # "Physical attack without a weapon",
    "phys_atk",
    # "Threats of physical attack with a weapon",
    "threat_phys_atk_weap",
    # "Threats of physical attack with a firearm or explosive device",
    "threat_phys_atk_fire_expl",
    # "Threats of physical attack without a weapon",
    "threat_phys_atk",
    # "Possession of a firearm or explosive device",
    "poss_fire_expl",
    # Number of Schools
    "total_schools",
    # "Percent of Schools Reporting"
    "total_schools_p",
    "C16",
]

In [87]:
# incidents = load_excel(
#     "offenses.xlsx",
#     names=SCHOOL_INCIDENT_COLUMN_MAPPINGS,
#     drop_columns=["C0", "C16"],
# )

incidents = pd.read_excel(
    BASE_PATH.joinpath( "offenses.xlsx"),
    # -- Exclude the top rows of formatting
    skiprows=[0, 1, 2, 3, 4],
    names=incident_col_mappings,
    # dtype={k: np.int64 for k in incident_col_mappings},
    # usecols=lambda x: x if x not in ["C0", "C16"] else None,
)

# -- Remove unused rows and columns
incidents.drop(["C0", "C16"], axis=1, inplace=True)
incidents.drop([52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62], inplace=True)

# -- Clean data types
incidents[incidents.select_dtypes(include=['float']).columns] = incidents.select_dtypes(include=['float']).astype(int)
incidents.head()

/Users/seanbaier/Documents/projects/mscs/Courses/machine-learning/intro-machine-learning/csca5622-final-project/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,state,rape_atmp_rape,sex_aslt,rob_weap,rob_fire_expl,rob,phys_atk_fight_weap,phys_atk_fight_fire_expl,phys_atk,threat_phys_atk_weap,threat_phys_atk_fire_expl,threat_phys_atk,poss_fire_expl,total_schools,total_schools_p
0,United States,394,9255,619,250,22580,11717,2777,787176,10350,3437,197896,5862,96360,98
1,Alabama,4,99,2,2,207,156,50,10100,152,52,2218,80,1400,100
2,Alaska,0,4,0,0,41,24,3,1693,27,7,262,11,503,100
3,Arizona,32,404,1,0,583,475,24,17436,460,75,6112,249,1977,100
4,Arkansas,0,30,4,9,174,197,76,9349,125,14,776,35,1092,100


In [27]:
# -- Merge all data on the State column

data = teacher_creds.merge(preschool, on="state").merge(incidents, on="state")

# -- Remove total of united states
data.drop([0], inplace=True) 

# -- Which state has the most the most number of schools. Choosing number here since most are 100%
most_schools = data[data['total_schools'] == data['total_schools'].max()]

# -- What's the school with the highest rate of meeting requirements?
highest_requirements_met = data[data['meeting_requirements_p'] == data['meeting_requirements_p'].max()]
highest_requirements_met

,state,classroom_teachers_fte,meeting_requirements_fte,meeting_requirements_p,first_year_teachers_fte,first_year_teachers_p,second_year_teachers_fte,second_year_teachers_p,total_schools_x,total_schools_p_x,...,rob,phys_atk_fight_weap,phys_atk_fight_fire_expl,phys_atk,threat_phys_atk_weap,threat_phys_atk_fire_expl,threat_phys_atk,poss_fire_expl,total_schools,total_schools_p
16,Iowa,35165.25,35163.25,99.9943,1666.1,4.7379,1775.5,5.04902,1424.0,100.0,...,149,125,1,14828,52,14,2364,40,1365.0,100.0


In [28]:
# plot a pair plot of the data using the seaborn library
cols_to_plot = data.columns
plt.figure(figsize=(16, 16))
sns.pairplot(votes[cols_to_plot], diag_kind="kde")

plt.savefig('election_pair_plot.png', dpi = 300, bbox_inches = 'tight')
# your code here

NameError: name 'plt' is not defined